In [1]:
import cv2
import time
import matplotlib.pyplot as plt
import numpy as np
import sys
import win32api, win32con, win32gui
from mss import mss

In [2]:
def random_sleep(sec):
    time.sleep(sec + 0.3*(np.random.rand()-0.5))

def get_image_difference(image_1, image_2):
    first_image_hist = cv2.calcHist([image_1], [0], None, [256], [0, 256])
    second_image_hist = cv2.calcHist([image_2], [0], None, [256], [0, 256])

    img_hist_diff = cv2.compareHist(first_image_hist, second_image_hist, cv2.HISTCMP_BHATTACHARYYA)
    img_template_probability_match = cv2.matchTemplate(first_image_hist, second_image_hist, cv2.TM_CCOEFF_NORMED)[0][0]
    img_template_diff = 1 - img_template_probability_match

    commutative_image_diff = (img_hist_diff / 10) + img_template_diff
    return commutative_image_diff

def find_img_pos(screen, img, count=1, dist=None, interval=5, verbose=False):
    H, W = screen.shape[0:2]
    h, w = img.shape[0:2]
    min_commutative_image_diff = 10000*np.ones([count])
    pos = np.zeros([count, 2], dtype=np.int32)
    all_pixel_num = (H-h)*(W-w)
    for i in range(0, H-h, interval):
        for j in range(0, W-w, interval):
            commutative_image_diff = get_image_difference(img, screen[i:i+h,j:j+w])
            idx = np.argmax(min_commutative_image_diff)
            if commutative_image_diff < min_commutative_image_diff[idx]:
                min_commutative_image_diff[idx] = commutative_image_diff
                pos[idx] = np.array([i, j], dtype=np.int32)
                        
            if verbose:
                current_pixel_num = i*(W-w)+j
                sys.stdout.write('\ron scanning... {:.2f}%'.format(current_pixel_num/all_pixel_num*100))
    return pos, min_commutative_image_diff

def get_screen(sct, monitor):
    screen = np.array(sct.grab(monitor))
    return screen

def click(pos):
    x, y = pos[1], pos[0]
    win32api.SetCursorPos((x, y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, x, y, 0, 0)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, x, y, 0, 0)
    
def drag(pos1, pos2):
    pos1 = pos1[::-1]
    pos2 = pos2[::-1]
    win32api.SetCursorPos(pos1)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, pos1[0], pos1[1], 0, 0); time.sleep(1)
    for i in np.arange(0, 1, 0.05):
        win32api.SetCursorPos(pos1 + ((pos2-pos1)*i).astype('int')); time.sleep(0.03)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, pos2[0], pos2[1], 0, 0)
    
def get_mouse_pos():
    _, _, pos = win32gui.GetCursorInfo()
    return np.array(pos)[::-1]

In [3]:
class NoxManager:
    def __init__(self, config):
        self.sct = mss()
        self.monitor = self.sct.monitors[0]
        self.nox_size = config['nox_size']
        self.nox_pos = self.get_nox_pos()
        self.nox_monitor = self.get_nox_monitor()
        
        self.get_main_menu_pos()
        
    def get_nox_pos(self):
        print('-------------------------------------')
        input('Run Nox Application and Press Any Key')
        print('Now Find Nox Position')
        screen = get_screen(self.sct, self.sct.monitors[0])
        nox_img = cv2.imread('./image_files/nox.PNG')
        nox_pos = find_img_pos(screen, nox_img, verbose=True)[0][0]
        
        print('\n')
        print('Refining Nox Position')
        i, j = nox_img.shape[0:2]
        if nox_pos[0] - i < 0: i = nox_pos[0]
        if nox_pos[1] - j < 0: j = nox_pos[1]
        around_screen = screen[nox_pos[0]-i:nox_pos[0]+2*nox_img.shape[0],
                               nox_pos[1]-j:nox_pos[1]+2*nox_img.shape[1]]
        nox_pos -= np.array([i, j])
        nox_pos += find_img_pos(around_screen, nox_img, interval=1, verbose=True)[0][0]
        nox_pos[0] += nox_img.shape[0]
        
        print('\n')
        print('Finished')
        
        return nox_pos
        
    def get_nox_monitor(self):
        nox_monitor = self.monitor.copy()
        nox_monitor['height'] = self.nox_size[0]
        nox_monitor['width'] = self.nox_size[1]
        nox_monitor['top'] = self.nox_pos[0]
        nox_monitor['left'] = self.nox_pos[1]
        return nox_monitor
        
    def get_relative_pos(self, img, count=1, dist=None):
        screen = get_screen(self.sct, self.nox_monitor)
        pos, img_diff = find_img_pos(screen, img, count=count, dist=dist)
        pos += (np.array(img.shape[0:2])/2).astype('int')
        return pos, img_diff
    
    def get_main_menu_pos(self):
        print()
        print('------------------------------------------')
        input('Run Girl\'s Frontier and Move to Main Menu')
        print('Now Find Menu Positions')
        print()
        self.combat_pos = self.get_relative_pos(cv2.imread('./image_files/combat.PNG'))[0][0]
        print('Compat Pos Obtained')
        self.restore_pos = self.get_relative_pos(cv2.imread('./image_files/restore.PNG'))[0][0]
        print('Restore Pos Obtained')
        self.formation_pos = self.get_relative_pos(cv2.imread('./image_files/formation.PNG'))[0][0]
        print('Formation Pos Obtained')
        print()
        print('Finished')
        
    def relative_drag(self, pos1, pos2):
        drag(self.nox_pos+pos1, self.nox_pos+pos2)
        
    def get_relative_mouse_pos(self):
        pos = get_mouse_pos() - self.nox_pos
        return pos
        
    def click_relative_pos(self, pos):
        noisy_pos = self.nox_pos + pos + np.random.randint(-5, 5, 2)
        click(noisy_pos)
        
    def restore(self):
        res_pos, res_diff = self.get_relative_pos(cv2.imread('./image_files/restore_warning.PNG'), 1)
        if res_diff[0] < 0.01:
            self.click_relative_pos(res_pos[0]); random_sleep(3)
            while True:
                add_pos = self.get_relative_pos(cv2.imread('./image_files/add_doll.PNG'))[0][0]
                self.click_relative_pos(add_pos); random_sleep(3)

                deadly_pos, deadly_diff = self.get_relative_pos(cv2.imread('./image_files/deadly.PNG'))
                if deadly_diff[0] > 0.02:
                    cancel_pos = self.get_relative_pos(cv2.imread('./image_files/cancel.PNG'))[0][0]
                    self.click_relative_pos(cancel_pos); random_sleep(3)
                    return_pos = self.get_relative_pos(cv2.imread('./image_files/return.PNG'))[0][0]
                    self.click_relative_pos(return_pos); random_sleep(3)
                    break

                self.click_relative_pos(deadly_pos[0]); random_sleep(3)

                res_confirm_pos = self.get_relative_pos(cv2.imread('./image_files/restore_confirm.PNG'))[0][0]
                self.click_relative_pos(res_confirm_pos); random_sleep(3)

                fast_res_pos = self.get_relative_pos(cv2.imread('./image_files/fast_restore.PNG'))[0][0]
                self.click_relative_pos(fast_res_pos); random_sleep(3)

                res_confirm_pos_ = self.get_relative_pos(cv2.imread('./image_files/restore_confirm_.PNG'))[0][0]
                self.click_relative_pos(res_confirm_pos_); random_sleep(3)

                close_pos = self.get_relative_pos(cv2.imread('./image_files/close.PNG'))[0][0]
                self.click_relative_pos(close_pos); random_sleep(3)
                
        self.click_relative_pos(np.array([396, 407])); random_sleep(1) # close pop-up

        
    def run_5_4n(self, repeat, fast_restore=True):
        print('------------------------------')
        print('Return to Base for Running 5-4')
        input('Delete Cafe Icon and Press Any Key')
        firstrun = input('Is EP.05 Activated? [Y/N] ') == 'N'
        print()
        
        points = np.array([[194, 222],
                           [195, 402],
                           [207, 612],
                           [370, 622],
                           [505, 612]])
        
        for epoch in range(repeat):
            self.click_relative_pos(self.combat_pos), random_sleep(3)
            
            if firstrun:
                ep_pos = self.get_relative_pos(cv2.imread('./image_files/EP05.PNG'))[0][0]
                self.click_relative_pos(ep_pos); random_sleep(3)
                firstrun = False
            
            stage_pos = self.get_relative_pos(cv2.imread('./image_files/EP05-4.PNG'))[0][0]
            self.click_relative_pos(stage_pos); random_sleep(3)
            
            normal_pos = self.get_relative_pos(cv2.imread('./image_files/normal.PNG'))[0][0]
            self.click_relative_pos(normal_pos); random_sleep(3)
            
            command_pos = self.get_relative_pos(cv2.imread('./image_files/command.PNG'))[0][0]
            self.click_relative_pos(command_pos); random_sleep(3)
            
            dummy_pos = self.get_relative_pos(cv2.imread('./image_files/dummy.PNG'))[0][0]
            self.click_relative_pos(dummy_pos); random_sleep(3)
            
            confirm_pos = self.get_relative_pos(cv2.imread('./image_files/confirm.PNG'))[0][0]
            self.click_relative_pos(confirm_pos); random_sleep(3)     

            self.relative_drag(np.array([400, 10]), np.array([400, 700])); random_sleep(3)
            
            heliport_pos = self.get_relative_pos(cv2.imread('./image_files/heilport.PNG'))[0][0]
            self.click_relative_pos(heliport_pos); random_sleep(3)
            
            self.click_relative_pos(confirm_pos); random_sleep(3)
            
            start_pos = self.get_relative_pos(cv2.imread('./image_files/start.PNG'))[0][0]
            self.click_relative_pos(start_pos); random_sleep(3)
            
            points_ = np.vstack([[heliport_pos], points])
            
            ## combat phase
            
            for i in range(points.shape[0]):
                self.click_relative_pos(points_[i]); random_sleep(3)
                self.click_relative_pos(points_[i+1]); random_sleep(5)
                if i == 3:
                    self.click_relative_pos(np.array([300, 400])); random_sleep(1)
                random_sleep(30)
                for _ in range(4):
                    self.click_relative_pos(np.array([300, 400])); random_sleep(1)
                random_sleep(3)
                
            terminate_pos = self.get_relative_pos(cv2.imread('./image_files/terminate.PNG'))[0][0]
            self.click_relative_pos(terminate_pos); random_sleep(15)
            for _ in range(4):
                self.click_relative_pos(np.array([300, 400])); random_sleep(1)
            random_sleep(3)
            
            self.click_relative_pos(np.array([396, 407])); random_sleep(1) # close pop-up
            
            self.restore()
            
            sys.stdout.write('\r{0}/{1}'.format(epoch+1, repeat))
            
    def run_4_3e(self, repeat, fast_restore=True):
        print('-------------------------------')
        print('Return to Base for Running 4-3E')
        input('Delete Cafe Icon and Press Any Key')
        firstrun = input('Is EP.04 Activated? [Y/N] ') == 'N'
        print()
        
        points = np.array([[270, 594],
                           [104, 655],
                           [449, 634],
                           [284, 539],
                           [ 61, 550]])
        
        for epoch in range(repeat):
            self.click_relative_pos(self.combat_pos), random_sleep(3)
            
            if firstrun:
                ep_pos = self.get_relative_pos(cv2.imread('./image_files/EP04.PNG'))[0][0]
                self.click_relative_pos(ep_pos); random_sleep(3)
                firstrun = False
            
            emerg_pos = self.get_relative_pos(cv2.imread('./image_files/emergency.PNG'))[0][0]
            self.click_relative_pos(emerg_pos); random_sleep(3)
            
            stage_pos = self.get_relative_pos(cv2.imread('./image_files/EP04-3E.PNG'))[0][0]
            self.click_relative_pos(stage_pos); random_sleep(3)
            
            normal_pos = self.get_relative_pos(cv2.imread('./image_files/normal.PNG'))[0][0]
            self.click_relative_pos(normal_pos); random_sleep(3)
            
            command_pos = self.get_relative_pos(cv2.imread('./image_files/command.PNG'))[0][0]
            self.click_relative_pos(command_pos); random_sleep(3)
            
            dummy_pos = self.get_relative_pos(cv2.imread('./image_files/dummy.PNG'))[0][0]
            self.click_relative_pos(dummy_pos); random_sleep(3)
            
            confirm_pos = self.get_relative_pos(cv2.imread('./image_files/confirm.PNG'))[0][0]
            self.click_relative_pos(confirm_pos); random_sleep(3)     

            self.relative_drag(np.array([300, 700]), np.array([300, 100])); random_sleep(3)
            
            heliport_pos = self.get_relative_pos(cv2.imread('./image_files/heilport.PNG'))[0][0]
            self.click_relative_pos(heliport_pos); random_sleep(3)
            
            self.click_relative_pos(confirm_pos); random_sleep(3)
            
            start_pos = self.get_relative_pos(cv2.imread('./image_files/start.PNG'))[0][0]
            self.click_relative_pos(start_pos); random_sleep(3)

            self.click_relative_pos(heliport_pos); random_sleep(3)
            self.click_relative_pos(points[0]); random_sleep(30)
            for _ in range(4):
                self.click_relative_pos(np.array([300, 400])); random_sleep(1)
            random_sleep(3)
            
            self.click_relative_pos(points[0]); random_sleep(3)
            self.click_relative_pos(points[1]); random_sleep(30)
            for _ in range(4):
                self.click_relative_pos(np.array([300, 400])); random_sleep(1)
            random_sleep(3)
            
            self.relative_drag(np.array([100, 400]), np.array([500, 400])); random_sleep(3)
            
            self.click_relative_pos(points[2]); random_sleep(3)
            self.click_relative_pos(points[3]); random_sleep(30)
            for _ in range(4):
                self.click_relative_pos(np.array([300, 400])); random_sleep(1)
            random_sleep(3)
            
            self.click_relative_pos(points[3]); random_sleep(3)
            self.click_relative_pos(points[4]); random_sleep(30)
            for _ in range(4):
                self.click_relative_pos(np.array([300, 400])); random_sleep(1)
            random_sleep(3)
                
            terminate_pos = self.get_relative_pos(cv2.imread('./image_files/terminate.PNG'))[0][0]
            self.click_relative_pos(terminate_pos); random_sleep(15)
            for _ in range(4):
                self.click_relative_pos(np.array([300, 400])); random_sleep(1)
            random_sleep(3)
            
            self.click_relative_pos(np.array([396, 407])); random_sleep(1) # close pop-up
            
            self.restore()
            
            sys.stdout.write('\r{0}/{1}'.format(epoch+1, repeat))

In [4]:
config = {
    'nox_size' : (600, 800)
}

In [5]:
manager = NoxManager(config=config)

-------------------------------------
Run Nox Application and Press Any Key
Now Find Nox Position
on scanning... 99.71%

Refining Nox Position
on scanning... 99.98%

Finished

------------------------------------------
Run Girl's Frontier and Move to Main Menu
Now Find Menu Positions

Compat Pos Obtained
Restore Pos Obtained
Formation Pos Obtained

Finished


In [9]:
manager.run_4_3e(repeat=15)

-------------------------------
Return to Base for Running 4-3E
Delete Cafe Icon and Press Any Key
Is EP.04 Activated? [Y/N] Y

1/15

KeyboardInterrupt: 